In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/bespoke/constants.py:448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propref_aux_target['ixx'] = list(range(len(propref_aux_target))) # dumb self ix so can grab sigmoid from within this smaller df


('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'u5imxujp'

In [3]:
wandb.init(id='b1xzj883', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

18293.759

In [5]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(["run_555a", "run_556a", "run_556b", "run_556c"], m, wandb=wandb)

CPU times: user 34.7 ms, sys: 26.2 s, total: 26.2 s
Wall time: 29.6 s


In [7]:
sd = torch.load(f"{BESPOKE_ROOT}/models/m.torch")
m = try_load_state_dict(m, sd)

loaded backbone.conv_stem.weight
loaded backbone.bn1.weight
loaded backbone.bn1.bias
loaded backbone.bn1.running_mean
loaded backbone.bn1.running_var
loaded backbone.bn1.num_batches_tracked
loaded backbone.blocks.0.0.conv_dw.weight
loaded backbone.blocks.0.0.bn1.weight
loaded backbone.blocks.0.0.bn1.bias
loaded backbone.blocks.0.0.bn1.running_mean
loaded backbone.blocks.0.0.bn1.running_var
loaded backbone.blocks.0.0.bn1.num_batches_tracked
loaded backbone.blocks.0.0.se.conv_reduce.weight
loaded backbone.blocks.0.0.se.conv_reduce.bias
loaded backbone.blocks.0.0.se.conv_expand.weight
loaded backbone.blocks.0.0.se.conv_expand.bias
loaded backbone.blocks.0.0.conv_pw.weight
loaded backbone.blocks.0.0.bn2.weight
loaded backbone.blocks.0.0.bn2.bias
loaded backbone.blocks.0.0.bn2.running_mean
loaded backbone.blocks.0.0.bn2.running_var
loaded backbone.blocks.0.0.bn2.num_batches_tracked
loaded backbone.blocks.0.1.conv_dw.weight
loaded backbone.blocks.0.1.bn1.weight
loaded backbone.blocks.0.1.bn1

In [8]:
opt = torch.optim.Adam(m.parameters(), lr=3e-4)

In [9]:
%%time
BS = 32
dataloader_trn = TrnLoader(bs=BS)

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


Got first chunk
CPU times: user 386 ms, sys: 3.82 s, total: 4.2 s
Wall time: 15.7 s


In [10]:
"""%%time
rw_evaluator.evaluate()"""

'%%time\nrw_evaluator.evaluate()'

In [11]:
model_stem = "1.23"

In [12]:
"""freeze_model(m, True)
unfreeze_part_of_model(m, "wps_head")
unfreeze_part_of_model(m, "aux_targets_head")
unfreeze_part_of_model(m, "obsnet")
[p.requires_grad for p in m.parameters()]"""

'freeze_model(m, True)\nunfreeze_part_of_model(m, "wps_head")\nunfreeze_part_of_model(m, "aux_targets_head")\nunfreeze_part_of_model(m, "obsnet")\n[p.requires_grad for p in m.parameters()]'

In [13]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator)

In [14]:
#trainer.reload_state()

In [ ]:
trainer.train()

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
